In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import os

In [3]:
from sklearn.metrics import r2_score,mean_squared_error

# Malhanobis Distance for sample weights

In [4]:
def malhanobis_dist(invcov,array):
    return float(1)/np.sqrt(np.dot(np.dot(array,invcov),np.transpose(array)))

# Linear Regression

In [5]:
os.chdir('../output_data')

In [6]:
df = pd.read_csv('train_v05.csv',header=0)

In [7]:
df.set_index('Id',inplace=True)

In [8]:
df.shape

(1168, 62)

In [9]:
invcov = np.linalg.inv(np.array(df.drop(['SalePrice'],axis=1).cov()))

In [10]:
x = np.transpose(np.array([[df[col]-df[col].mean()] for col in df.drop(['SalePrice'],axis=1).columns]))[:,0,:]

In [11]:
x.shape

(1168, 61)

In [12]:
sample_weights = np.array([malhanobis_dist(invcov,x[i,:]) for i in range(x.shape[0])])

In [13]:
sample_weights.shape[0]

1168

In [14]:
True in np.isinf(sample_weights) or True in np.isnan(sample_weights)

False

In [15]:
os.chdir('../pickles')

In [16]:
pickle.dump(sample_weights,open('inferential_sample_weights.pickle','wb'))

# Inferential Model

In [17]:
scale = pickle.load(open('scale.pickle','rb'))

In [18]:
pca = pickle.load(open('pca.pickle','rb'))

In [19]:
x = scale.transform(df.drop(['SalePrice'],axis=1))

In [20]:
model = LinearRegression()

In [21]:
y = np.array(df['SalePrice'])

In [22]:
model.fit(x,y,sample_weight=sample_weights)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [23]:
y_pred = model.predict(x)

In [24]:
r2_train = r2_score(y,y_pred)

In [25]:
r2_train

0.80342597048089415

In [26]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [27]:
evaluating_criteria

0.037364972094947747

In [28]:
os.chdir('../Models/Inferential_models')

In [29]:
pickle.dump(model,open('LR.pickle','wb'))

In [30]:
model_columns = df.drop(['SalePrice'],axis=1).columns

In [31]:
os.chdir('../../pickles')

In [32]:
pickle.dump(model_columns,open('model_columns.pickle','wb'))

# Predictive Model

In [33]:
os.chdir('../output_data')

In [34]:
df_pca = pd.read_csv('train_pca.csv',header=0)

In [35]:
df_pca.set_index('Id',inplace=True)

In [36]:
invcov = np.linalg.inv(np.array(df_pca.drop(['SalePrice'],axis=1).cov()))

In [38]:
x = np.transpose(np.array([[df_pca[col]-df_pca[col].mean()] for col in df_pca.drop(['SalePrice'],axis=1).columns]))[:,0,:]

In [39]:
x.shape

(1168, 47)

In [40]:
sample_weights = np.array([malhanobis_dist(invcov,x[i,:]) for i in range(x.shape[0])])

In [41]:
sample_weights.shape[0]

1168

In [42]:
True in np.isinf(sample_weights) or True in np.isnan(sample_weights)

False

In [43]:
os.chdir('../pickles')

In [44]:
pickle.dump(sample_weights,open('predictive_sample_weights.pickle','wb'))

In [45]:
y=np.array(df_pca['SalePrice'])

In [46]:
model = LinearRegression()
model.fit(np.array(df_pca.drop(['SalePrice'],axis=1)),y,sample_weight=sample_weights)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
os.chdir('../Models/pca_models')

In [48]:
pickle.dump(model,open('LR.pickle','wb'))

In [49]:
y_pred = model.predict(np.array(df_pca.drop(['SalePrice'],axis=1)))

In [50]:
r2_train = r2_score(y,y_pred)

In [51]:
r2_train

0.78582344432901685

In [52]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [53]:
evaluating_criteria

0.041750120823309193

# Random Forest

# Inferential Model

In [54]:
os.chdir('../../pickles')

In [55]:
sample_weights = pickle.load(open('inferential_sample_weights.pickle','rb'))

In [56]:
os.chdir('../Models/Inferential_models')

In [57]:
tuned_parameters={'max_features':[None,'sqrt','log2'],'min_samples_leaf':[i*.01 for i in range(1,11)]}
model = GridSearchCV(RandomForestRegressor(n_jobs=-1,n_estimators=1000,oob_score=True,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=1000)

In [58]:
x = scale.transform(df.drop(['SalePrice'],axis=1))

In [59]:
y = np.array(df['SalePrice'])

In [61]:
model.fit(x,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.737963 -   3.5s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.685026 -   3.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.9s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.620546 -   3.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.6s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.731158 -   3.6s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   14.2s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 

[CV]  max_features=None, min_samples_leaf=0.08, score=0.481871 -   3.3s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  2.1min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.592234 -   3.2s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  2.2min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.569349 -   3.1s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.2min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.461638 -   3.0s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  2.3min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.446378 -   3.1s
[Parallel(n_jobs=1)]: Done  42 out o

[CV]  max_features=sqrt, min_samples_leaf=0.05, score=0.520470 -   3.2s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  4.0min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.454177 -   3.0s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  4.0min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.447833 -   3.0s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  4.1min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.371575 -   3.1s
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:  4.1min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.480170 -   3.0s
[Parallel(n_jobs=1)]: Done  79 out o

[CV]  max_features=log2, min_samples_leaf=0.03, score=0.508781 -   3.0s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:  5.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.428406 -   3.0s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed:  5.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.549173 -   3.0s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed:  6.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.557901 -   3.0s
[Parallel(n_jobs=1)]: Done 115 out of 115 | elapsed:  6.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.04 ........................
[CV]  max_features=log2, min_samples_leaf=0.04, score=0.478220 -   3.1s
[Parallel(n_jobs=1)]: Done 116 out o

[CV]  max_features=log2, min_samples_leaf=0.1, score=0.318831 -   3.0s
[Parallel(n_jobs=1)]: Done 149 out of 149 | elapsed:  7.8min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=0.318891 -   3.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  7.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  7.8min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=1000)

In [62]:
model.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.01, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [63]:
model.best_score_

0.6920251236524096

In [64]:
final_model = RandomForestRegressor(n_estimators=1000,min_samples_leaf=.01,random_state=42,oob_score=True)

In [65]:
final_model.fit(x,y,sample_weight=sample_weights)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.01, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [66]:
y_pred = final_model.predict(x)

In [67]:
r2_score(y,y_pred)

0.76685795274309121

In [68]:
pickle.dump(final_model,open('RF.pickle','wb'))

In [69]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [70]:
evaluating_criteria

0.032449684408266947

# Predictive Model

In [71]:
os.chdir('../../pickles')

In [72]:
sample_weights = pickle.load(open('predictive_sample_weights.pickle','rb'))

In [73]:
os.chdir('../Models/pca_models')

In [74]:
x = np.array(df_pca.drop(["SalePrice"],axis=1))

In [75]:
y =np.array(df_pca['SalePrice'])

In [76]:
tuned_parameters={'max_features':[None,'sqrt','log2'],'min_samples_leaf':[i*.01 for i in range(1,11)]}
model = GridSearchCV(RandomForestRegressor(n_jobs=-1,n_estimators=1000,oob_score=True,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=1000)

In [77]:
model.fit(x,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.771585 -   7.7s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.707295 -   8.8s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.5s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.606235 -   7.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   24.1s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 ........................
[CV]  max_features=None, min_samples_leaf=0.01, score=0.750949 -   7.8s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   31.9s remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.01 

[CV]  max_features=None, min_samples_leaf=0.08, score=0.461103 -   4.6s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  3.7min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.635261 -   4.6s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  3.8min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.08 ........................
[CV]  max_features=None, min_samples_leaf=0.08, score=0.590847 -   4.8s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  3.9min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.631454 -   4.4s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  3.9min remaining:    0.0s
[CV] max_features=None, min_samples_leaf=0.09 ........................
[CV]  max_features=None, min_samples_leaf=0.09, score=0.596351 -   4.4s
[Parallel(n_jobs=1)]: Done  42 out o

[CV]  max_features=sqrt, min_samples_leaf=0.05, score=0.412053 -   3.3s
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  6.0min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.362271 -   3.1s
[Parallel(n_jobs=1)]: Done  76 out of  76 | elapsed:  6.1min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.350170 -   3.1s
[Parallel(n_jobs=1)]: Done  77 out of  77 | elapsed:  6.1min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.275761 -   3.1s
[Parallel(n_jobs=1)]: Done  78 out of  78 | elapsed:  6.2min remaining:    0.0s
[CV] max_features=sqrt, min_samples_leaf=0.06 ........................
[CV]  max_features=sqrt, min_samples_leaf=0.06, score=0.380216 -   3.3s
[Parallel(n_jobs=1)]: Done  79 out o

[CV]  max_features=log2, min_samples_leaf=0.03, score=0.433040 -   3.2s
[Parallel(n_jobs=1)]: Done 112 out of 112 | elapsed:  8.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.339154 -   3.2s
[Parallel(n_jobs=1)]: Done 113 out of 113 | elapsed:  8.0min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.451367 -   3.3s
[Parallel(n_jobs=1)]: Done 114 out of 114 | elapsed:  8.1min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.03 ........................
[CV]  max_features=log2, min_samples_leaf=0.03, score=0.453685 -   3.2s
[Parallel(n_jobs=1)]: Done 115 out of 115 | elapsed:  8.1min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.04 ........................
[CV]  max_features=log2, min_samples_leaf=0.04, score=0.394841 -   3.2s
[Parallel(n_jobs=1)]: Done 116 out o

[CV]  max_features=log2, min_samples_leaf=0.1, score=0.262456 -   3.0s
[Parallel(n_jobs=1)]: Done 149 out of 149 | elapsed:  9.9min remaining:    0.0s
[CV] max_features=log2, min_samples_leaf=0.1 .........................
[CV]  max_features=log2, min_samples_leaf=0.1, score=0.259199 -   3.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 10.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 10.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=1000)

In [78]:
model.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.01, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [79]:
model.best_score_

0.7071598064081132

In [80]:
final_model = model.best_estimator_

In [81]:
final_model.fit(x,y,sample_weight=sample_weights)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.01, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [82]:
pickle.dump(final_model,open('RF.pickle','wb'))

In [83]:
y_pred = final_model.predict(x)

In [84]:
r2_score(y,y_pred)

0.81082568186144677

In [85]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [86]:
evaluating_criteria

0.02684995345944062

# Gradient Boosting Model

# Inferential Model

In [87]:
os.chdir('../../pickles')

In [88]:
sample_weights= pickle.load(open('inferential_sample_weights.pickle','rb'))

In [89]:
os.chdir('../Models/Inferential_models')

In [90]:
x = scale.transform(df.drop(['SalePrice'],axis=1))

In [91]:
y = np.array(df['SalePrice'])

In [94]:
tuned_parameters={'n_estimators':[100*i for i in range(1,11)]}
model = GridSearchCV(GradientBoostingRegressor(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=50)

In [95]:
model.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.801202 -   0.6s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.743740 -   0.6s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.714440 -   0.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.784200 -   0.6s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[CV] n_estimators=100 .............................

[CV] ....................... n_estimators=800, score=0.714936 -   4.6s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  1.6min remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.776097 -   4.5s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  1.7min remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.702417 -   4.6s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.7min remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.798779 -   5.2s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  1.8min remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.742039 -   5.2s
[Parallel(n_jobs=1)]: Done  42 out of  42

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=50)

In [96]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [97]:
tuned_parameters={'n_estimators':[20*i for i in range(1,6)]}
model = GridSearchCV(GradientBoostingRegressor(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=50)

In [98]:
model.fit(x,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.753930 -   0.1s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.711256 -   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.652336 -   0.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.738553 -   0.1s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.5s remaining:    0.0s
[CV] n_estimators=20 ...............................

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 40, 60, 80, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=50)

In [99]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=80, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [102]:
tuned_parameters={'max_depth':[8,10,12,15,20],'min_samples_leaf':[.01*i for i in range(1,11)],'subsample':[.8,.9,1]
                 ,'max_features':[None,'sqrt','log2']}
model = GridSearchCV(GradientBoostingRegressor(n_estimators=80,learning_rate=.1,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=2500)

In [103]:
model.fit(x,y)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.795444 -   0.5s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.747475 -   0.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.9s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.711954 -   0.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.786840 -   0

[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.714153 -   0.3s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   13.0s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.785932 -   0.3s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   13.3s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.716565 -   0.4s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   13.7s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.795281 -   0.4s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   14.0s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.785468 -   0.3s
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed:   22.9s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.732972 -   0.3s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:   23.2s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.683643 -   0.3s
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed:   23.5s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.781774 -   0.3s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:   23.8s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.763533 -   0.2s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:   31.0s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.705985 -   0.2s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   31.2s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.773005 -   0.2s
[Parallel(n_jobs=1)]: Done 101 out of 101 | elapsed:   31.4s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.698956 -   0.2s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed:   31.6s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_l

[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.691144 -   0.2s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:   38.1s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.628444 -   0.2s
[Parallel(n_jobs=1)]: Done 133 out of 133 | elapsed:   38.3s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.764592 -   0.2s
[Parallel(n_jobs=1)]: Done 134 out of 134 | elapsed:   38.5s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.692032 -   0.2s
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   38.7s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=

...

[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.03, score=0.680804 -   0.1s
[Parallel(n_jobs=1)]: Done 2133 out of 2133 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.03, score=0.795890 -   0.1s
[Parallel(n_jobs=1)]: Done 2134 out of 2134 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.03, score=0.734177 -   0.1s
[Parallel(n_jobs=1)]: Done 2135 out of 2135 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.03, score=0.772227 -   0.1s
[Parallel(n_jobs=1)]: Done 2136 out of 2136 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, m

[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.05, score=0.736933 -   0.1s
[Parallel(n_jobs=1)]: Done 2165 out of 2165 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05, score=0.771163 -   0.1s
[Parallel(n_jobs=1)]: Done 2166 out of 2166 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05, score=0.701633 -   0.0s
[Parallel(n_jobs=1)]: Done 2167 out of 2167 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.05, score=0.648802 -   0.1s
[Parallel(n_jobs=1)]: Done 2168 out of 2168 | elapsed:  5.5min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, m

[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.07, score=0.618510 -   0.1s
[Parallel(n_jobs=1)]: Done 2198 out of 2198 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.07, score=0.765126 -   0.1s
[Parallel(n_jobs=1)]: Done 2199 out of 2199 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.07, score=0.714582 -   0.1s
[Parallel(n_jobs=1)]: Done 2200 out of 2200 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.07 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.07, score=0.766978 -   0.0s
[Parallel(n_jobs=1)]: Done 2201 out of 2201 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_dep

[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.09, score=0.687271 -   0.1s
[Parallel(n_jobs=1)]: Done 2230 out of 2230 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09, score=0.753844 -   0.0s
[Parallel(n_jobs=1)]: Done 2231 out of 2231 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09, score=0.703104 -   0.0s
[Parallel(n_jobs=1)]: Done 2232 out of 2232 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.09, score=0.609388 -   0.0s
[Parallel(n_jobs=1)]: Done 2233 out of 2233 | elapsed:  5.6min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, m

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=80, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'subsample': [0.8, 0.9, 1], 'max_depth': [8, 10, 12, 15, 20], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=2500)

In [104]:
model.best_score_

0.776206111351987

In [105]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=42, subsample=1, verbose=0, warm_start=False)

In [112]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=15,max_features='log2',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=40,learning_rate=.2),tuned_parameters,cv=5,scoring='r2')

In [113]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=40, presort='auto',
             random_state=42, subsample=1, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [114]:
model.best_score_

0.7427207365877507

In [115]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=15,max_features='log2',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=160,learning_rate=.05),tuned_parameters,cv=5,scoring='r2')

In [117]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=160,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [118]:
model.best_score_

0.7700799711657773

In [119]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=15,max_features='log2',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=320,learning_rate=.025),tuned_parameters,cv=5,scoring='r2')

In [120]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.025, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=320,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [121]:
model.best_score_

0.7752144208237569

In [122]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=15,max_features='log2',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=800,learning_rate=.01),tuned_parameters,cv=5,scoring='r2')

In [123]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=800,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [124]:
model.best_score_

0.7758644779880383

In [125]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=15,max_features='log2',min_samples_leaf=.01,subsample=1,random_state=42,n_estimators=640,learning_rate=.0125),tuned_parameters,cv=5,scoring='r2')

In [126]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0125, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=640,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [127]:
model.best_score_

0.7771340051951469

In [128]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0125, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=640,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False)

In [129]:
final_model = model.best_estimator_

In [130]:
final_model.fit(x,y,sample_weight=sample_weights)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0125, loss='ls', max_depth=15,
             max_features='log2', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.01, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=640,
             presort='auto', random_state=42, subsample=1, verbose=0,
             warm_start=False)

In [131]:
pickle.dump(final_model,open('GBM.pickle','wb'))

In [132]:
y_pred = final_model.predict(x)

In [133]:
r2_score(y,y_pred)

0.90614438005329478

In [134]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [135]:
evaluating_criteria

0.012916928417264727

# Predictive Model

In [136]:
os.chdir('../../pickles')

In [137]:
sample_weights = pickle.load(open('predictive_sample_weights.pickle','rb'))

In [138]:
x = np.array(df_pca.drop(['SalePrice'],axis=1))

In [139]:
y = np.array(df_pca['SalePrice'])

In [140]:
os.chdir('../Models/pca_models')

In [141]:
tuned_parameters={'n_estimators':[100*i for i in range(1,11)]}
model = GridSearchCV(GradientBoostingRegressor(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=50)

In [142]:
model.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.746832 -   1.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.726923 -   1.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.651886 -   1.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.9s remaining:    0.0s
[CV] n_estimators=100 ................................................
[CV] ....................... n_estimators=100, score=0.762905 -   1.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.9s remaining:    0.0s
[CV] n_estimators=100 .............................

[CV] ....................... n_estimators=800, score=0.650766 -   7.4s
[Parallel(n_jobs=1)]: Done  38 out of  38 | elapsed:  2.6min remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.755146 -   7.6s
[Parallel(n_jobs=1)]: Done  39 out of  39 | elapsed:  2.8min remaining:    0.0s
[CV] n_estimators=800 ................................................
[CV] ....................... n_estimators=800, score=0.692863 -   7.4s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  2.9min remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.715832 -   8.8s
[Parallel(n_jobs=1)]: Done  41 out of  41 | elapsed:  3.0min remaining:    0.0s
[CV] n_estimators=900 ................................................
[CV] ....................... n_estimators=900, score=0.730208 -   8.1s
[Parallel(n_jobs=1)]: Done  42 out of  42

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=50)

In [143]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [144]:
tuned_parameters={'n_estimators':[20*i for i in range(1,6)]}
model = GridSearchCV(GradientBoostingRegressor(learning_rate=.1,min_samples_leaf=.01,max_depth=10,subsample=.8,
                                               random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=50)

In [145]:
model.fit(x,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.767144 -   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.692852 -   0.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.626010 -   0.2s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[CV] n_estimators=20 .................................................
[CV] ........................ n_estimators=20, score=0.741341 -   0.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s
[CV] n_estimators=20 ...............................

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 40, 60, 80, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=50)

In [146]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.01,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=40, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [147]:
tuned_parameters={'max_depth':[8,10,12,15,20],'min_samples_leaf':[.01*i for i in range(1,11)],'subsample':[.8,.9,1]
                 ,'max_features':[None,'sqrt','log2']}
model = GridSearchCV(GradientBoostingRegressor(n_estimators=40,learning_rate=.1,random_state=42),tuned_parameters,cv=5,scoring='r2',verbose=2500)

In [148]:
model.fit(x,y)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.751540 -   0.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.707621 -   0.3s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.647313 -   0.3s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.1s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.01, score=0.751552 -   0

[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.648423 -   0.3s
[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:   12.4s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.772223 -   0.3s
[Parallel(n_jobs=1)]: Done  34 out of  34 | elapsed:   12.6s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=0.03, score=0.702771 -   0.3s
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:   12.9s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.03, score=0.783692 -   0.3s
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   13.2s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.756106 -   0.2s
[Parallel(n_jobs=1)]: Done  66 out of  66 | elapsed:   21.2s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.699168 -   0.2s
[Parallel(n_jobs=1)]: Done  67 out of  67 | elapsed:   21.4s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.632775 -   0.2s
[Parallel(n_jobs=1)]: Done  68 out of  68 | elapsed:   21.6s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.05, score=0.760944 -   0.2s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:   21.9s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min

[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.737600 -   0.2s
[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:   28.5s remaining:    0.0s
[CV] max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=0.9, max_depth=8, min_samples_leaf=0.07, score=0.659589 -   0.2s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   28.7s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.747769 -   0.2s
[Parallel(n_jobs=1)]: Done 101 out of 101 | elapsed:   28.9s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.07, score=0.687771 -   0.2s
[Parallel(n_jobs=1)]: Done 102 out of 102 | elapsed:   29.1s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_l

[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.687152 -   0.2s
[Parallel(n_jobs=1)]: Done 132 out of 132 | elapsed:   34.8s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.617604 -   0.2s
[Parallel(n_jobs=1)]: Done 133 out of 133 | elapsed:   35.0s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.733389 -   0.2s
[Parallel(n_jobs=1)]: Done 134 out of 134 | elapsed:   35.2s remaining:    0.0s
[CV] max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09 
[CV]  max_features=None, subsample=1, max_depth=8, min_samples_leaf=0.09, score=0.681918 -   0.2s
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   35.3s remaining:    0.0s
[CV] max_features=None, subsample=0.8, max_depth=8, min_samples_leaf=

...

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.582342 -   0.1s
[Parallel(n_jobs=1)]: Done 2143 out of 2143 | elapsed:  4.9min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.712647 -   0.1s
[Parallel(n_jobs=1)]: Done 2144 out of 2144 | elapsed:  4.9min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.03, score=0.685836 -   0.1s
[Parallel(n_jobs=1)]: Done 2145 out of 2145 | elapsed:  4.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.04 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.04, score=0.710755 -   0.1s
[Parallel(n_jobs=1)]: Done 2146 out of 2146 | elapsed:  4.9min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=2

[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06, score=0.514506 -   0.1s
[Parallel(n_jobs=1)]: Done 2178 out of 2178 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06, score=0.573655 -   0.1s
[Parallel(n_jobs=1)]: Done 2179 out of 2179 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06 
[CV]  max_features=log2, subsample=0.8, max_depth=20, min_samples_leaf=0.06, score=0.581256 -   0.1s
[Parallel(n_jobs=1)]: Done 2180 out of 2180 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.06 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.06, score=0.700239 -   0.1s
[Parallel(n_jobs=1)]: Done 2181 out of 2181 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, m

[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08, score=0.664394 -   0.1s
[Parallel(n_jobs=1)]: Done 2211 out of 2211 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08, score=0.606105 -   0.1s
[Parallel(n_jobs=1)]: Done 2212 out of 2212 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08, score=0.496254 -   0.0s
[Parallel(n_jobs=1)]: Done 2213 out of 2213 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08 
[CV]  max_features=log2, subsample=0.9, max_depth=20, min_samples_leaf=0.08, score=0.558709 -   0.0s
[Parallel(n_jobs=1)]: Done 2214 out of 2214 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=0.9, m

[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1, score=0.653688 -   0.0s
[Parallel(n_jobs=1)]: Done 2246 out of 2246 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1, score=0.635442 -   0.0s
[Parallel(n_jobs=1)]: Done 2247 out of 2247 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1, score=0.507252 -   0.0s
[Parallel(n_jobs=1)]: Done 2248 out of 2248 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1 
[CV]  max_features=log2, subsample=1, max_depth=20, min_samples_leaf=0.1, score=0.667911 -   0.0s
[Parallel(n_jobs=1)]: Done 2249 out of 2249 | elapsed:  5.0min remaining:    0.0s
[CV] max_features=log2, subsample=1, max_depth=20, min_sample

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=40, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'sqrt', 'log2'], 'subsample': [0.8, 0.9, 1], 'max_depth': [8, 10, 12, 15, 20], 'min_samples_leaf': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=2500)

In [149]:
model.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=20, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.02,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=40, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False)

In [150]:
model.best_score_

0.7326265101591277

In [151]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=20,learning_rate=.2),tuned_parameters,cv=5,scoring='r2')

In [152]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=20, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=0.02,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=20, presort='auto', random_state=42,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [153]:
model.best_score_

0.7152186542240286

In [157]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=80,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=80,learning_rate=.05),tuned_parameters,cv=5,scoring='r2')

In [158]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='ls', max_depth=80,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=80, presort='auto',
             random_state=42, subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [159]:
model.best_score_

0.7305616904827673

In [160]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=160,learning_rate=.025),tuned_parameters,cv=5,scoring='r2')

In [161]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.025, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=160,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [162]:
model.best_score_

0.7300020976611322

In [163]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=320,learning_rate=.0125),tuned_parameters,cv=5,scoring='r2')

In [164]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.0125, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=320,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [165]:
model.best_score_

0.7301016653352883

In [166]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=400,learning_rate=.01),tuned_parameters,cv=5,scoring='r2')

In [167]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=400,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [168]:
model.best_score_

0.7309117276840568

In [169]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=640,learning_rate=.00625),tuned_parameters,cv=5,scoring='r2')

In [170]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.00625, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=640,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [171]:
model.best_score_

0.7310705551116519

In [172]:
tuned_parameters={}
model = GridSearchCV(GradientBoostingRegressor(max_depth=20,min_samples_leaf=.02,subsample=.8,random_state=42,n_estimators=1280,learning_rate=.003125),tuned_parameters,cv=5,scoring='r2')

In [173]:
model.fit(x,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.003125, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1280,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=0)

In [174]:
model.best_score_

0.7312136305353195

In [175]:
final_model = model.best_estimator_

In [176]:
final_model.fit(x,y,sample_weight=sample_weights)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.003125, loss='ls', max_depth=20,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.02, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1280,
             presort='auto', random_state=42, subsample=0.8, verbose=0,
             warm_start=False)

In [177]:
pickle.dump(final_model,open('GBM.pickle','wb'))

In [178]:
y_pred = final_model.predict(x)

In [180]:
r2_score(y,y_pred)

0.88274564244764331

In [181]:
evaluating_criteria = mean_squared_error(np.apply_along_axis(np.log,0,y),np.apply_along_axis(np.log,0,y_pred))

In [182]:
evaluating_criteria

0.015406143620626247